# DEWAN LAB MICROENDOSCOPE ANALYSIS TEMPLATE
##### This script uses functions from the Inscopix API to: 1) preprocess video files (i.e., bandpass filtering and downsampling), 2) motion correct, 3) identify cells using CNMFE, 4) process videos for dF/F, 5) produce a max projection image, and 6) export files. 
#### Copy this script into a folder specific for the animal/session, rename it according to the animal/session (e.g., VGAT4_Session1_InscopixAnalysis), and adjust user configurables

### Step 1: Import Modules

In [1]:
# We will remove this when we are done developing the script
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import isx  #Inscopix API; Navigate to C:\Program Files\Inscopix\Data Processing\ ; run pip intsall -e .
from Python.Helpers import DewanIOhandler

### Step 2: Adjust User Configurables

In [14]:
experiment_name = 'VGAT4-Session1'
data_directory = None # Overwrite if not using local folder
output_directory = None

# Set to True to delete all the intermediary Inscopix Files
cleanup_interim_files = False

# Set to True to output a .mp4 version of the recording. Note: This takes a long time.
export_video_file = False

focus_planes = [700]  
# Likely will not need this anymore, we will get it automatically.
# If these values do not match the efocus used in the exp, code will crash

#Downsample Settings
spatial_downsample_factor = 4 # Factor of 4 is recommended for CNMFE

#Spatial Bandpass Settings
spatial_bp_low_cutoff = 0.005
spatial_bp_high_cutoff = 0.500

#Motion Correction Settings
max_translation = 20
motion_c_low_bandpass_cutoff = None
motion_c_high_bandpass_cutoff = None

#CNMFE Settings
cell_diameter = 7
merge_threshold = 0.7 #default is 0.7, use default when you have a 10-min video for baseline
gaussian_kernel_size = 3
output_unit_type = 0
minimum_correlation = 0.9
minimum_pnr = 15

### Step 3: Get directories and video files

In [16]:
if data_directory is None:
    data_directory = "./InscopixProcessing/RawData"

if output_directory is None:
    output_directory = "./InscopixProcessing/DataAnalysis"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

if not os.path.exists(data_directory):
    os.makedirs(data_directory)

proc_move_files = []
proc_cs_files = []

video_base, video_files, gpio_input_path = DewanIOhandler.get_project_files(data_directory)

IndexError: list index out of range

### Austin - Delete once we verify that deinterleaving is no longer necessary

In [40]:
# series_rec_names = {}
#
# for i, focus_plane in enumerate(focus_planes):
#     deinterlaced_files = [file for file in DewanIOhandler.make_isx_path(video_files, output_directory, str(focus_plane))]
#     key_name = f'focal{str(focus_plane)}'
#     series_rec_names[key_name] = deinterlaced_files
#
# series_rec_names_keys = list(series_rec_names.keys())

# Don't think we need any of this anymore. Documentation says deinterleaving is now automatic during preprocessing

#deinterlaced_files = np.reshape(deinterlaced_files_matrix, (1, len(focus_planes) * trials))

# if len(focus_planes) > 1:
# isx.deinterleave(video_files, deinterlaced_files, focus_planes)
# if len(focus_planes) > 1:
#     deinterlaced_files_matrix = np.array((len(focus_planes), trials))
#     for i in range(len(focus_planes)):
#         deinterlaced_files_matrix[i, :] = isx.make_output_file_paths(video_files, output_directory, str(focus_planes(i)))
#     deinterlaced_files = np.reshape(deinterlaced_files_matrix, (1, len(focus_planes) * trials))
#
#     isx.deinterleave(video_files, deinterlaced_files, focus_planes)
#
#     for i in range(len(focus_planes)):
#         keyName = f'focal{focus_planes[i]}'
# else:
#     keyName = f'focal{focus_planes[0]}'
#     series_rec_names[keyName] = video_files
#     series_rec_names_keys = list(series_rec_names.keys())


['R:\\Inscopix_Projects\\VGAT5-Odor-Identity-Expt\\2022-11-16-13-35-25_video.isxd'] 1


### Step 4: Process video files for standard analysis

In [ ]:
#     Austin - do we need these notes? 
#     for key in range(len(series_rec_names_keys)):
#     series_name = series_rec_names_keys[key]
#     rec_files = series_rec_names[series_name]
#     first_file = video_files[key]

In [ ]:

# STEP 4A: PREPROCESS VIDEO FILES 
post_process_files = DewanIOhandler.make_isx_path(video_files, output_directory, 'PP')
if not DewanIOhandler.check_files(post_process_files):
    print('Starting Preprocessing...')
    isx.preprocess(video_files, post_process_files, spatial_downsample_factor=spatial_downsample_factor)

# STEP 4B: BANDPASS FILTER VIDEO FILES
bandpass_files= DewanIOhandler.make_isx_path(post_process_files, output_directory, 'BP')
if not DewanIOhandler.check_files(list(bandpass_files)):
    print('Starting Bandpass Filtering...')
    isx.spatial_filter(post_process_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)

# STEP 4C.1: GENERATE MEAN IMAGE AS REFERENCE FRAME FOR THE MOTION CORRECTION
mean_projection_file = os.path.join(output_directory, f'{video_base}-mean_image.isxd')
if not DewanIOhandler.check_files(mean_projection_file):
    print('Creating Mean Image...')
    isx.project_movie(bandpass_files, mean_projection_file, stat_type='mean')

# STEP 4C.2: MOTION CORRECT VIDEO FILES
motion_correction_files = DewanIOhandler.make_isx_path(bandpass_files, output_directory, 'MC')
translation_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'translations', 'csv')
crop_rect_file = os.path.join(output_directory, f'{video_base}-crop_rect.csv')

if not DewanIOhandler.check_files(motion_correction_files):
    print('Starting Motion Correction...')
    isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file,
                       low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                       output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)

# STEP 4D: PROCESS VIDEO FILES FOR DELTA F/F 
dff_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'DFF')
if not DewanIOhandler.check_files(dff_files):
    print('Calculating DF/F...')
    isx.dff(motion_correction_files, dff_files, f0_type='mean')

# STEP 4E: MAKE A MAX PROJECTION FILE OF THE CELLS (NOTE IT IS DOWNSAMPLED)
max_projection_file = DewanIOhandler.make_isx_path(dff_files, output_directory, 'MAX_PROJ')

for i, each in enumerate(max_projection_file):  
    if not DewanIOhandler.check_files(max_projection_file):
        start_time = time.strftime("%H:%M:%S")
        print(f"{start_time} Starting Max Projection Creation. This might take a while...")
        isx.project_movie(dff_files[i], each, stat_type='max')
        end_time = time.strftime("%H:%M:%S")
        print(f"{end_time} Max Projection Creation finished. That took a while...")

tiff_path = DewanIOhandler.make_isx_path(max_projection_file, output_directory, addition=None, extention='.tiff')

# STEP 4E.2: Convert .isx max projection files to tiff files
for i, each in enumerate(tiff_path):
    isx.export_isxd_image_to_tiff(max_projection_file[i], each)


# STEP 4F.1: RUN CNFME
cnmfe_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'CNMFE')
if not DewanIOhandler.check_files(motion_correction_files) or not DewanIOhandler.check_files(cnmfe_files):
    print('Running CNMFe (This Will Take a Long Time)...')
    start_time = time.time()
    isx.run_cnmfe(motion_correction_files, cnmfe_files, output_directory, cell_diameter=cell_diameter, merge_threshold=merge_threshold, gaussian_kernel_size=gaussian_kernel_size,
                  output_unit_type='df', min_corr=minimum_correlation, min_pnr=minimum_pnr, processing_mode='parallel_patches', num_threads=8)  #  No DF/F on CNMFE Files
    end_time = time.time()
    print(f'CNMFE Finished! Total Elapsed Time: {(end_time - start_time) / 60} (min)')

# STEP 4F.2: APPLY CELL CONTOURS
contour_files = DewanIOhandler.make_isx_path(dff_files, output_directory, 'AC')

if not DewanIOhandler.check_files(dff_files) or not DewanIOhandler.check_files(cnmfe_files) or not DewanIOhandler.check_files(contour_files):
    print('Applying Cell Contours...')
    isx.apply_cell_set(dff_files, cnmfe_files[0], contour_files, 0)

# STEP 4G: EXPORT FILES
tiff_file_name = f'{video_base}.tiff'
csv_file_name = f'{video_base}.csv'
json_file_name = f'{video_base}-CONTOURS.json'
props_file_name = f'{video_base}-Properties.csv'
#gpio_input = f'{video_base}.gpio'
gpio_output_filename = f'{video_base}-GPIO.csv'

tiff_image_path = os.path.join(output_directory, tiff_file_name)
trace_file_path = os.path.join(output_directory, csv_file_name)
props_file_path = os.path.join(output_directory, props_file_name)

print('Exporting Traces and Props...')
isx.export_cell_set_to_csv_tiff(contour_files, trace_file_path, tiff_image_path, time_ref='unix', output_props_file= props_file_path)

json_file_path = os.path.join(output_directory, json_file_name)

print('Exporting Cell Contours...')
isx.export_cell_contours(cnmfe_files[0], json_file_path)

# gpio_input_path = os.path.join(data_directory, gpio_input)
gpio_output_path = os.path.join(output_directory, gpio_output_filename)

print("Exporting GPIO to csv...")
isx.export_gpio_set_to_csv(gpio_input_path, gpio_output_path, time_ref='unix')

# STEP 4H (OPTIONAL): EXPORT MP4 VIDEO OF RECORDING
if export_video_file:
    movie_filename = f'{video_base}.mp4'
    movie_file_path = os.path.join(output_directory, movie_filename)
    print('Exporting Movie to MP4...')
    isx.export_movie_to_mp4(dff_files, movie_file_path, compression_quality= 0.4, write_invalid_frames= True)
    
print('Processing Finished!')

if cleanup_interim_files:
    for each in post_process_files, bandpass_files, motion_correction_files, dff_files, cnmfe_files:
        os.remove(each)


### STEP 5 (OPTIONAL): Produce high resolution images/videos

In [ ]:
# STEP 5A: PREPROCESS VIDEO FILES 
post_process_files = DewanIOhandler.make_isx_path(video_files, output_directory, 'HR-PP')
if not DewanIOhandler.check_files(post_process_files):
    print('Starting Preprocessing...')
    isx.preprocess(video_files, post_process_files)

# STEP 5B: BANDPASS FILTER VIDEO FILES
bandpass_files= DewanIOhandler.make_isx_path(post_process_files, output_directory, 'HR-BP')
if not DewanIOhandler.check_files(list(bandpass_files)):
    print('Starting Bandpass Filtering...')
    isx.spatial_filter(post_process_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)

# STEP 5C.1: GENERATE MEAN IMAGE AS REFERENCE FRAME FOR THE MOTION CORRECTION
mean_projection_file = os.path.join(output_directory, f'{video_base}-HR-mean_image.isxd')
if not DewanIOhandler.check_files(mean_projection_file):
    print('Creating Mean Image...')
    isx.project_movie(bandpass_files, mean_projection_file, stat_type='mean')

# STEP 5C.2: MOTION CORRECT VIDEO FILES
motion_correction_files = DewanIOhandler.make_isx_path(bandpass_files, output_directory, 'HR-MC')
translation_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'translations', 'csv')
crop_rect_file = os.path.join(output_directory, f'{video_base}-HR-crop_rect.csv')

if not DewanIOhandler.check_files(motion_correction_files):
    print('Starting Motion Correction...')
    isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file,
                       low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                       output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)

# STEP 5D: PROCESS VIDEO FILES FOR DELTA F/F 
dff_files = DewanIOhandler.make_isx_path(motion_correction_files, output_directory, 'HR-DFF')
if not DewanIOhandler.check_files(dff_files):
    print('Calculating DF/F...')
    isx.dff(motion_correction_files, dff_files, f0_type='mean')

#### STEP 6 (OPTIONAL): Produce a high resolution max projection image

In [ ]:
# STEP 6.1: MAKE A MAX PROJECTION FILE OF THE CELLS
max_projection_file = DewanIOhandler.make_isx_path(dff_files, output_directory, 'HR-MAX_PROJ')

for i, each in enumerate(max_projection_file):  
    if not DewanIOhandler.check_files(max_projection_file):
        start_time = time.strftime("%H:%M:%S")
        print(f"{start_time} Starting Max Projection Creation. This might take a while...")
        isx.project_movie(dff_files[i], each, stat_type='max')
        end_time = time.strftime("%H:%M:%S")
        print(f"{end_time} Max Projection Creation finished. That took a while...")

tiff_path = DewanIOhandler.make_isx_path(max_projection_file, output_directory, addition='HR', extention='.tiff')

# STEP 6.2: Convert .isx max projection files to tiff files
for i, each in enumerate(tiff_path):
    isx.export_isxd_image_to_tiff(max_projection_file[i], each)

#### STEP 7 (OPTIONAL): Produce a high resolution dF/F movie

In [ ]:
movie_filename = f'{video_base}-HR.mp4'
movie_file_path = os.path.join(output_directory, movie_filename)
print('Exporting Movie to MP4...')
isx.export_movie_to_mp4(dff_files, movie_file_path, compression_quality= 0.4, write_invalid_frames= True)